In [ ]:
# LUCKNOW EV CHARGING DATASET PIPELINE
### Notebook 1A + Notebook 1B + Notebook 2 Combined
'''
This notebook constructs the *complete dataset* for EV charging infrastructure optimization in Lucknow.

---

## CONTENTS
1. **Notebook 1A — Spatial Data**
   - Road network (OSMnx)
   - POIs
   - Building footprints
   - 500m grid candidate sites

2. **Notebook 1B — Synthetic Temporal Demand**
   - Zone creation (commercial / residential / peripheral)
   - Transfer-learning arrival curves (from UrbanEV/Shenzhen)
   - Per-cell λ_i(t) for low/base/high scenarios

3. **Notebook 2 — Composite Spatial Demand Modeling**
   - POI density
   - Building-area density
   - Normalized weights
   - demand_score
   - Maps + histograms

---

Outputs saved in `output/`:
- lucknow_graph.graphml
- pois_lucknow.geojson
- buildings_lucknow.geojson
- candidate_grid_500m.geojson
- candidate_sites_with_weights.geojson
- zone_arrival_profiles.png
- scenario_arrival_profiles.png
- synthetic_temporal_demand_per_cell.npz
- synthetic_temporal_demand_summary.csv
- demand_histogram.png
- candidate_demand_map.png
'''


In [3]:
!pip install osmnx geopy rtree shapely fiona geopandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 7.7 MB/s eta 0:00:00


In [4]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.cluster import KMeans
import osmnx as ox

plt.rcParams["figure.figsize"] = (10, 6)

OUT = "output"
os.makedirs(OUT, exist_ok=True)

print("Output directory:", OUT)


Output directory: output


In [5]:
place = "Lucknow, Uttar Pradesh, India"

G = ox.graph_from_place(place, network_type="drive")
G = ox.project_graph(G)
ox.save_graphml(G, f"{OUT}/lucknow_graph.graphml")
print("Saved graph.")


Saved graph.


In [7]:
tags = {"amenity": True, "shop": True, "office": True}
pois = ox.features_from_place(place, tags=tags)   # NEW correct function
pois = pois.to_crs(G.graph["crs"])
pois = pois[pois.geometry.type.isin(["Point","Polygon","MultiPolygon"])]
pois.to_file(f"{OUT}/pois_lucknow.geojson", driver="GeoJSON")
print("Saved POIs:", len(pois))


Saved POIs: 1757


In [9]:
# Try modern OSMnx method
try:
    buildings = ox.features_from_place(place, tags={"building": True})
except Exception as e:
    print("features_from_place failed. Error:", e)
    buildings = None

# If buildings still None, fallback to geocode + mask from entire place
if buildings is None or len(buildings) == 0:
    print("Fallback: Using geometries_from_bbox instead (broad but safe).")
    bounds = ox.geocode_to_gdf(place).to_crs(G.graph["crs"]).geometry.iloc[0].bounds
    north, south, east, west = bounds[3], bounds[1], bounds[2], bounds[0]
    buildings = ox.features_from_bbox(north, south, east, west, tags={"building": True})
    buildings = buildings.to_crs(G.graph["crs"])

# Keep only valid polygon geometries
buildings = buildings[buildings.geometry.type.isin(["Polygon", "MultiPolygon"])]

# Save
buildings.to_file(f"{OUT}/buildings_lucknow.geojson", driver="GeoJSON")
print("Saved buildings:", len(buildings))


Saved buildings: 8089


In [10]:
boundary_poly = ox.geocode_to_gdf(place).to_crs(G.graph["crs"]).geometry.iloc[0]
minx, miny, maxx, maxy = boundary_poly.bounds

xs = np.arange(minx, maxx, 500)
ys = np.arange(miny, maxy, 500)

points = [Point(x, y) for x in xs for y in ys]
candidates = gpd.GeoDataFrame(geometry=points, crs=G.graph["crs"])
candidates = candidates[candidates.intersects(boundary_poly)]

candidates.to_file(f"{OUT}/candidate_grid_500m.geojson", driver="GeoJSON")
print("Saved candidate grid:", len(candidates))


Saved candidate grid: 10100


In [11]:
cand = gpd.read_file(f"{OUT}/candidate_grid_500m.geojson")
n_cells = len(cand)
cand = cand.reset_index().rename(columns={"index":"id"})


In [12]:
coords = np.vstack([cand.geometry.x, cand.geometry.y]).T
kmeans = KMeans(n_clusters=3, random_state=42).fit(coords)
cand["zone_id"] = kmeans.labels_
zone_map = {0:"Z1_commercial", 1:"Z2_residential", 2:"Z3_peripheral"}
cand["zone"] = cand["zone_id"].map(zone_map)


In [13]:
hours = np.arange(24)

def norm(arr):
    arr = np.array(arr, float)
    arr[arr<0]=0
    return arr / (arr.sum()+1e-9)

lambda_Z1 = norm([0.2,0.2,0.2,0.3,0.4,0.7,1.0,1.1,1.0,0.9,0.7,0.6,
                  0.5,0.5,0.6,0.8,1.0,1.1,0.9,0.7,0.5,0.4,0.3,0.2])

lambda_Z2 = norm([0.1,0.1,0.1,0.1,0.2,0.3,0.4,0.6,0.7,0.9,1.0,1.1,
                  1.1,1.0,0.9,0.8,0.7,0.6,0.4,0.3,0.2,0.1,0.1,0.1])

lambda_Z3 = norm([0.1]*6 + [0.2]*3 + [0.3]*9 + [0.2]*3 + [0.1]*3)

profiles = {"Z1_commercial":lambda_Z1, "Z2_residential":lambda_Z2, "Z3_peripheral":lambda_Z3}


In [14]:
plt.figure(figsize=(10,6))
plt.plot(hours, lambda_Z1, label="Z1 commercial")
plt.plot(hours, lambda_Z2, label="Z2 residential")
plt.plot(hours, lambda_Z3, label="Z3 peripheral")
plt.xlabel("Hour")
plt.ylabel("Normalized Arrival Intensity")
plt.title("Stylized Arrival Curves (Transferred)")
plt.legend()
plt.grid(True)
plt.savefig(f"{OUT}/zone_arrival_profiles.png", dpi=200, bbox_inches="tight")
plt.close()


In [15]:
# demand proxy: uniform for now, replaced later by demand_score from Notebook 2
cand["s_i"] = 1.0

rho_vals = {"low":0.3, "base":1.0, "high":2.0}
lambda_cells = {k: np.zeros((n_cells,24)) for k in rho_vals}

for idx,row in cand.iterrows():
    p = profiles[row.zone]
    for scen,rho in rho_vals.items():
        lambda_cells[scen][idx,:] = rho * row.s_i * p

np.savez_compressed(
    f"{OUT}/synthetic_temporal_demand_per_cell.npz",
    hours=hours, **{f"lambda_{k}":v for k,v in lambda_cells.items()}
)

print("Saved temporal demand arrays.")


Saved temporal demand arrays.


In [16]:
plt.figure(figsize=(10,6))
for scen, arr in lambda_cells.items():
    city_profile = arr.sum(axis=0)
    plt.plot(hours, city_profile, label=scen)
plt.legend()
plt.title("Scenario-based Synthetic Arrival Profiles")
plt.xlabel("Hour")
plt.ylabel("Total Expected Arrivals")
plt.grid(True)
plt.savefig(f"{OUT}/scenario_arrival_profiles.png", dpi=200, bbox_inches="tight")
plt.close()


In [17]:
pois = gpd.read_file(f"{OUT}/pois_lucknow.geojson")
buildings = gpd.read_file(f"{OUT}/buildings_lucknow.geojson")

# CRS alignment
cand = cand.to_crs(pois.crs)


/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 212882546 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 101225610 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


In [18]:
cand["poi_count"] = 0
sidx = pois.sindex

for i,row in cand.iterrows():
    buf = row.geometry.buffer(300)
    possible = list(sidx.intersection(buf.bounds))
    cand.loc[i, "poi_count"] = pois.iloc[possible].intersects(buf).sum()


In [20]:
# Ensure everything is in a metric CRS
cand = cand.to_crs(epsg=3857)
buildings = buildings.to_crs(epsg=3857)

# Clean and compute building area
cand["bldg_area"] = 0.0
b_sidx = buildings.sindex

for i, row in cand.iterrows():
    buf = row.geometry.buffer(300)   # 300m radius
    possible = list(b_sidx.intersection(buf.bounds))
    if possible:
        subset = buildings.iloc[possible]
        subset = subset[subset.intersects(buf)]
        area = subset.area.sum()  # NOW correctly in square meters
    else:
        area = 0
    cand.loc[i, "bldg_area"] = area


In [21]:
def mm(s):
    return (s - s.min()) / (s.max() - s.min() + 1e-9)

cand["P_n"] = mm(cand["poi_count"])
cand["B_n"] = mm(cand["bldg_area"])

# weights
wB, wP = 1.0, 2.0

cand["demand_score"] = wB*cand["B_n"] + wP*cand["P_n"]

cand.to_file(f"{OUT}/candidate_sites_with_weights.geojson", driver="GeoJSON")
print("Saved weighted candidate file.")


Saved weighted candidate file.


In [22]:
plt.figure(figsize=(10,6))
plt.hist(cand["demand_score"], bins=30)
plt.title("Distribution of demand_score")
plt.xlabel("demand_score")
plt.ylabel("count")
plt.grid(True)
plt.savefig(f"{OUT}/demand_histogram.png", dpi=200, bbox_inches="tight")
plt.close()


In [23]:
plt.figure(figsize=(10,8))
cand.plot(column="demand_score", cmap="viridis", markersize=15, legend=True)
plt.title("Candidate Locations Colored by demand_score")
plt.axis("off")
plt.savefig(f"{OUT}/candidate_demand_map.png", dpi=200, bbox_inches="tight")
plt.close()


<Figure size 1000x800 with 0 Axes>